In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('ds4-which-whisk/test.csv')
train = pd.read_csv('ds4-which-whisk/train.csv')

In [16]:
train.shape

(2476, 7)

In [42]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0


In [4]:
train = train.dropna()

In [5]:
X_train = train.drop(columns=['category'])

In [6]:
y_train = train['category']

In [7]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0


In [88]:
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
from xgboost import XGBRegressor

rfc = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
rfr = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1)
xgb = XGBRegressor(n_estimators=2000, n_jobs=-1)
vect = TfidfVectorizer(stop_words='english')

In [10]:
pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [11]:
pipe.fit(train['description'], train['category'])

C:\Users\nchib\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [12]:
##train/test split on train/csv
##use the test.csv to make predictions
##Kaggle ID and category columns are integers


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1980, 6), (496, 6), (1980,), (496,))

In [33]:
pipe.fit(X_train['description'], y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [23]:
pipe.predict(['Monarch, the 75th anniversary limited edition of Canadaâ€™s best-selling whisky, raises the already high Crown Royal flavor bar. Zesty rye from an ancient Coffey still is the throbbing heart of this blend, balancing cloves, ginger, cinnamon, glowing hot pepper, and that gorgeous sour bitterness of rye grain against crispy, fresh-sawn lumber, fragrant lilacs, dark fruits, and green apples. Butterscotch, chocolate, toffee, mint, pine needles, and sweet pitchy balsam enrich a luscious, creamy mouthfeel carefully tempered by grapefruit pith. Editor'])

array([4.])

In [36]:
print('Validation Accuracy', pipe.score(X_test['description'], y_test))

Validation Accuracy 0.8306451612903226


In [43]:
pred = pipe.predict(test['description'])

In [ ]:
# y_pred = pipe.predict(X_test)

# # Write submission csv file
# submission = sample_submission.copy()
# submission['status_group'] = y_pred
# submission.to_csv('submission-04.csv', index=False)

In [44]:
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [45]:
submission.head()

,id,category
0,955,1
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [46]:
submission.to_csv('ds4-which-whisk/submission1.csv', index=False)

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (0.02, 0.05),
    'vect__max_features': (100, 500, 1000),
    'rfc__n_estimators': (20, 100, 400)
}

In [49]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4)

In [50]:
gs.fit(train['description'], train['category'])

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'rfc__n_estimators': (20, 100, 400)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
gs.best_score_

0.8945880452342488

In [52]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [53]:
embeddings = [nlp(doc).vector for doc in train['description']]

In [54]:
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [55]:
rfc.fit(embeddings, train['category'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [59]:
rfc.score(embeddings, train['category'])

1.0

In [57]:
embeddings_test = [nlp(doc).vector for doc in test['description']]

In [61]:
pred = rfc.predict(embeddings_test)

In [62]:
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [63]:
submission.head()

,id,category
0,955,2
1,3532,1
2,1390,1
3,1024,1
4,1902,1


In [64]:
submission.to_csv('ds4-which-whisk/submission2.csv', index=False)

In [90]:
pipe = Pipeline([('vect', vect), ('rfr', rfr)])

In [91]:
pipe.fit(X_train['description'], y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...imators=1000, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [94]:
print('Validation Accuracy', gs.score(X_test['description'], y_test))

Validation Accuracy 1.0


In [95]:
pred = gs.predict(test['description'])
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')
submission.head()
submission.to_csv('ds4-which-whisk/submission4.csv', index=False)

In [96]:
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [120]:
def tokenize(doc):
    
    d = nlp(doc)
    tokens = []
    
    
    # You can extract anything you want from Spacy to use as a feature
    # Noun Chunks aren't the end all be all. 
    # Try Extracting just tokens, or just adjectives, or just pronouns
    
    for chunk in d.noun_chunks:
        tokens.append(chunk.lemma_)
        
#     for token in tokens:
#         if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
#             tokens.remove(token.lemma_)
        
    return tokens

In [123]:
tokens

NameError: name 'tokens' is not defined

In [121]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(analyzer=tokenize, max_df=0.9, min_df=0.1)

In [122]:
vect.fit(train['description'])

CountVectorizer(analyzer=<function tokenize at 0x000002854215C730>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.9,
        max_features=None, min_df=0.1, ngram_range=(1, 1),
        preprocessor=None, stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [104]:
vect.get_feature_names()

['-PRON-',
 'a hint',
 'caramel',
 'cinnamon',
 'honey',
 'the finish',
 'the nose',
 'the palate',
 'vanilla',
 'water']

In [110]:
X = vect.transform(train['description'])

In [111]:
rfc.fit(X, train['category'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [113]:
X_test = vect.transform(X_test['description'])

In [114]:
print('Validation Accuracy', rfc.score(X_test, y_test))

Validation Accuracy 0.7237903225806451


In [116]:
from collections import Counter

def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(temp, columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc.sort_values(by='rank')

In [ ]:
wc = count(df['tokens'])